In [1]:
# Imports

import pandas as pd
import numpy as np 
import re
from sklearn.naive_bayes import CategoricalNB
from sklearn.preprocessing import LabelEncoder

In [2]:
root_df = pd.read_csv("data.csv")
root_df

,Message,Target
0,Esqueci minha senha,Recuperação de Senha
1,Não estou conseguindo realizar o pagamento da ...,Pagamento
2,"Boa tarde, não estou conseguindo pagar o bolet...",Pagamento
3,naum da pra paga,Pagamento
4,Não consigo fazer o pix da rematrícula,Pagamento
5,Quero acessar a aba de rematrícula,Rematrícula
6,como faz pra achar a aba pra rematrícular,Rematrícula
7,Deu erro pra terminar minha rematricula,Rematrícula
8,vou reprovar por faltas ?,Consulta de faltas
9,quantas faltas precisa pra repetir de ano ?,Consulta de faltas


In [3]:
# Cleaning stop-words

clean_df = root_df.copy()

#Removing digits
clean_df["Message"] = clean_df["Message"].apply(lambda x : re.sub('\d+', '', x))

stop_words = pd.read_csv("stop-words.txt").squeeze()

for word in stop_words:
    clean_df["Message"] = clean_df["Message"].apply(lambda x : x.replace(" " + word + " ", ' '))

clean_df["Message"]

0                                         Esqueci senha
1             Não conseguindo realizar pagamento fatura
2         Boa tarde, conseguindo pagar boleto desse mês
3                                         naum pra paga
4                     Não consigo fazer pix rematrícula
5                       Quero acessar a aba rematrícula
6             como faz pra achar a aba pra rematrícular
7                     Deu erro pra terminar rematricula
8                                 vou reprovar faltas ?
9              quantas faltas precisa pra repetir ano ?
10    Faltei mês inteiro, onde posso entrar conferir...
11    Falta saber senha pra conseguir fazer rematríc...
12    quando tento entrar área aluno aparece senha i...
13         abasfoiafawsfawfiowarfawoirwa aeaweaweaweawe
14                              O trabalho ruim, odiei 
15                        Minha rematricula aparecendo 
16                                         não funciona
17                            não gosto dessa fa

In [4]:
# Defining Key-Words

access_keywords =  ["acesso", "acessar", "entrar", "logar", "entro", "entrei", "acessado", "acesei", "acessei", "portal"]
reenroll_keywords = ["rematricula", "rematrícula", "rematicula", "matricular"]
find_keywords = ["achar", "achando", "procurei", "procurando", "buscando", "busquei"]
password_keywords = ["senha", "senia", "chave", "senhe", "password", "passe"]
feedback_keywords = ["odiei", "adorei", "bom", "gostei", "fraco", "excelente", "não ajudou", "ruim", "odiei", "lixo"]
payment_keywords = ["pagar", "paguei", "pix", "boleto", "pagamento", "pago", "extrato", "dinheiro", "fatura"]
absence_keywords = ["faltei", "faltas", "presença", "faltando", "faltou"]

keywords_set = {
    "Acess" : access_keywords,
    "Password" : password_keywords,
    "Find" : find_keywords,
    "Reenroll" : reenroll_keywords,
    "Feedback" : feedback_keywords,
    "Payment" : payment_keywords,
    "Absence" : absence_keywords
}

In [5]:
# Extracting features
def check_words(word, wordArr):
    return 1 if any(w in word for w in wordArr) else 0

def extract_features(input_df):
    new_df = input_df.copy()

    for key, keyword in keywords_set.items():
        new_df[key] = new_df["Message"].apply(lambda word : check_words(word.lower(), keyword))

    return new_df


df = extract_features(clean_df)
df

,Message,Target,Acess,Password,Find,Reenroll,Feedback,Payment,Absence
0,Esqueci senha,Recuperação de Senha,0,1,0,0,0,0,0
1,Não conseguindo realizar pagamento fatura,Pagamento,0,0,0,0,0,1,0
2,"Boa tarde, conseguindo pagar boleto desse mês",Pagamento,0,0,0,0,0,1,0
3,naum pra paga,Pagamento,0,0,0,0,0,0,0
4,Não consigo fazer pix rematrícula,Pagamento,0,0,0,1,0,1,0
5,Quero acessar a aba rematrícula,Rematrícula,1,0,0,1,0,0,0
6,como faz pra achar a aba pra rematrícular,Rematrícula,0,0,1,1,0,0,0
7,Deu erro pra terminar rematricula,Rematrícula,0,0,0,1,0,0,0
8,vou reprovar faltas ?,Consulta de faltas,0,0,0,0,0,0,1
9,quantas faltas precisa pra repetir ano ?,Consulta de faltas,0,0,0,0,0,0,1


In [6]:
# Encode Target

le = LabelEncoder()
df["Target"] = le.fit_transform(df["Target"])

df

,Message,Target,Acess,Password,Find,Reenroll,Feedback,Payment,Absence
0,Esqueci senha,3,0,1,0,0,0,0,0
1,Não conseguindo realizar pagamento fatura,2,0,0,0,0,0,1,0
2,"Boa tarde, conseguindo pagar boleto desse mês",2,0,0,0,0,0,1,0
3,naum pra paga,2,0,0,0,0,0,0,0
4,Não consigo fazer pix rematrícula,2,0,0,0,1,0,1,0
5,Quero acessar a aba rematrícula,4,1,0,0,1,0,0,0
6,como faz pra achar a aba pra rematrícular,4,0,0,1,1,0,0,0
7,Deu erro pra terminar rematricula,4,0,0,0,1,0,0,0
8,vou reprovar faltas ?,0,0,0,0,0,0,0,1
9,quantas faltas precisa pra repetir ano ?,0,0,0,0,0,0,0,1


In [7]:
# Training

X = df.drop(columns=["Message", "Target"])
Y = df["Target"]

clf = CategoricalNB(force_alpha=True)

clf.fit(X, Y)

result = clf.predict(X[2:3])
print(le.inverse_transform(result))

['Pagamento']


In [8]:
import joblib

filename = 'saved_models/trained_model.sav'
joblib.dump(clf, filename)

joblib.dump(le, 'saved_models/label_encoder.sav')

['saved_models/label_encoder.sav']

In [9]:
loaded_model = joblib.load(filename)

message = input()

my_df = pd.DataFrame(data={ message }, columns=["Message"]) 
my_df = extract_features(my_df)

sample = my_df.drop(columns="Message")

result = loaded_model.predict(sample)

print(le.inverse_transform(result)[0])

Não Compreendível
